# Parse Q Data
Start by importing some utilities and the raw data:

In [1]:
import rawData, json, jupyterImport
jupyterImport.enableJupyterImports()
from Utilities import find, genSamples, genSample, pluck, copyKeys, combineDicts

In [2]:
def extractRawReports(data=rawData.q()):
  rawReportsDict = combineDicts(map(lambda s: s['reports'], data))
  for reportId, rawReport in rawReportsDict.iteritems():
    rawReport['reportId'] = reportId
  return rawReportsDict.values()

For some reason, Q reports don't include term / year anywhere on the pages. We have to scrape this data separately and then merge it with the reports:

In [3]:
def addTermYears(reports):
  origins = {}
  for key, termDict in rawData.qCourseIds().iteritems():
    for deptList in termDict.values():
      for courseId in deptList:
        term = key.split('_')[1]
        year = key.split('_')[0]
        term = 'Fall' if term == '1' else 'Spring'
        year = year if term == 'Fall' else str(int(year) + 1)
        origins[courseId] = {'term': term, 'year': year}
  for report in reports:
    if report['reportId'] in origins:
      report.update(origins[report['reportId']])

In [4]:
import q_map
def parseReportsAndComments(rawReports):
  comments = {}
  reports = []
  for rawReport in rawReports:
    report = {}
    for mapper in q_map.mappers:
      mapper.map(rawReport, to=report)
    reports.append(report)
    q_map.comments.map(rawReport, to=comments)
  addTermYears(reports)
  return (reports, comments)

In [5]:
reports, comments = parseReportsAndComments(extractRawReports())

In [6]:
### Run to inspect
genSample(reports)

{'evals': 32,
 'group': u'MATH',
 'number': u'1A',
 'profs': [{'displayName': u'Oliver Knill',
   'matchName': u'oliver knill',
   'responses': {'accessible': {'score': 4.75},
    'discussion': {'score': 4.27},
    'enthusiasm': {'score': 4.66},
    'feedback': {'score': 4.29},
    'instructor': {'score': 4.06},
    'lectures': {'score': 4.31},
    'turnaround': {'score': 4.61}}}],
 'reasons': {'concentration': u'62%',
  'elective': u'22%',
  'expos': u'3%',
  'genEd': u'28%',
  'preMed': u'12%',
  'secondary': u'6%'},
 'reportId': u'29716',
 'responses': {'assignments': {'score': 3.72},
  'feedback': {'score': 3.87},
  'materials': {'score': 3.87},
  'overall': {'score': 4.06},
  'recommend': {'score': 3.75},
  'section': {'score': 4.0},
  'workload': {'score': 6.16}},
 'size': 38,
 'term': 'Spring',
 'title': u'Introduction to Calculus',
 'year': '2012'}